In [2]:
import csv
import numpy as np
import datetime
import matplotlib.pyplot as plt
# class Death:
#     def __init__(self, state, date, death_type, data_value):
#         self.state = state
#         self.date = date
#         self.death_type = death_type
#         self.data_value = data_value

    
    
#ORDER of CSV below
# 0 ﻿State
# 1 Year
# 2 Month
# 3 Period
# 4 Indicator
# 5 Data Value
# 6 Percent Complete
# 7 Percent Pending Investigation
# 8 State Name
# 9 Footnote
# 10 Footnote Symbol
# 11 Predicted Value
Deaths = []
states = np.array([])
max_death = 0
#OPIOID DEATH INDICATOR IS- "Opioids (T40.0-T40.4,T40.6)"
with open('State Level Drug Death Data.csv') as csvfile: 
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        if row[4].find("Opioids (T40.0-T40.4,T40.6)") > -1:
            long_month_name = row[2]
            datetime_object = datetime.datetime.strptime(long_month_name, "%B")
            month_number = datetime_object.month
            date = datetime.date(int(row[1]), month_number,1)
            #order is state name, date, type of death, and number of deaths
            #print(row[5])
            if(row[5] == ''):
                row[5] = '0'
            row[5] = row[5].replace(",","")
#             if(row[8] == "Utah"):
#                 print(row[5])
            death = [row[8], date,"Opioid", int(row[5])]
            if(death[3] > max_death and row[8].find("United") < 0):
                max_death = death[3]
#                 if(max_death > 13000):
# #                     print(death[0])
            if(len(states) == 0 or row[8] not in states):
                states = np.append(states,row[8])
            Deaths.append(death)
Deaths = np.array(Deaths)
#AT THIS POINT, ALL OPIOID DEATHS SHOULD HAVE BEEN READ IN
#print(states)
#we want to sort by state, then date. i.e we want states to be together
Deaths = Deaths[Deaths[:,1].argsort()] # First sort doesn't need to be stable.
Deaths = Deaths[Deaths[:,0].argsort(kind='mergesort')]
states.sort()
#print(Deaths)

#isolate by location. 
i = 0
#fig, axs = plt.subplots(len(states))
fig, axs = plt.subplots(len(states))
state = 0

while(i < len(Deaths)-1):
    time = []
    deaths = []
    while(i < len(Deaths)-1 and Deaths[i][0] == Deaths[i+1][0]): #checks that states are equal
        time.append(Deaths[i][1])
        deaths.append(Deaths[i][3])
        i = i+1
    time.append(Deaths[i][1])
    deaths.append(Deaths[i][3])
#     plt.plot(time,deaths)
#     plt.xlabel("Date")
#     plt.ylabel("Trailing 12 month deaths")
#     plt.scatter(time,deaths)
#     plt.yticks(np.arange(0, max(deaths), step=max(deaths)/5))
#     plt.title(states[state])
#     plt.figure()
#    print(states[state],Deaths[i][0])
#     axs[state].set_title(states[state])
#     axs[state].plot(time, deaths)
    if(states[state].find("United") >= 0):
        axs[state].set_ylim(0, max(deaths) + 50)
    else:
        axs[state].set_ylim(0, max_death + 50)
   # axs[state].set_xlim(datetime.date(2018,5,1), datetime.date(2020,3,1))
    state = state + 1
    i = i+1
plt.subplots_adjust(left=None, bottom=-len(states)/2, right=None, top=len(states)-len(states)/2, wspace=None, hspace=0.3)
plt.show()

<Figure size 640x480 with 43 Axes>

In [3]:
import csv
import numpy as np
import datetime
import matplotlib.pyplot as plt

# 0 "Notes"
# 1 "Underlying Cause of death"
# 2 "Underlying Cause of death Code"
# 3 "State"
# 4 "State Code"
# 5 "County"
# 6 "County Code"
# 7 "Place of Death"
# 8 "Place of Death Code"
# 9 Deaths
# 10 Population
# 11 Crude Rate

f = open("Cause Of Death Count.txt", "r")
Deaths = []
for i,x in enumerate(f):
    s = x.split('\t')
#     for j,s in enumerate(split):
#         print(j,s.replace('"',''))
    # State, County, per capita deaths
    if(i == 0 or s[9].find("Not") > -1):
        continue
    s[5] = s[5].replace('"','')
    s[5] = s[5].replace('.','')
    d = [s[3].replace('"',''), s[5], int(s[9].replace('"',''))]
    Deaths.append(d)
Deaths = np.array(Deaths)
Deaths = Deaths[Deaths[:,2].argsort()]
Deaths = Deaths[Deaths[:,0].argsort(kind='mergesort')]
Deaths = Deaths[Deaths[:,1].argsort(kind='mergesort')]
#print(Deaths)
county = []
deaths = []
for d in Deaths:
    if len(county) == 0 or county[len(county)-1] != d[1]:
        county.append(d[1])
        deaths.append(int(d[2]))
    else:
        #print(deaths[len(county)-1])
        deaths[len(county)-1] = int(deaths[len(county)-1]) + int(d[2])
        #print(deaths[len(county)-1])
#print(deaths)

#gotta match counties now:
census_counties = []
census_pop = []
with open('2019Census_Population_Data_by_County.csv') as csvfile: 
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        row[0] = row[0].replace('.','')
        #print(row[0])
#         if(row[0].find('St. Louis') >= 0):
#             print(row[0])
        census_counties.append(row[0])
        #print(row)
        census_pop.append(int(row[len(row)-1].replace(',','')))
for i,c in enumerate(county):
    for j,cc in enumerate(census_counties):
#         if(c.find('St. Louis')>=0 and cc.find('St. Louis') >= 0):
#             print(c,cc)
        if c.split()[0] == cc.split()[0] and c.split()[1] == cc.split()[1] and c.split()[2][0:1] == cc.split()[2][0:1]:
            deaths[i] = deaths[i]*census_pop[j]/100000 #county death data is deaths per 100,000
D = []
for i,c in enumerate(county):
    D.append([county[i],int(deaths[i])])
#D = np.array(D)
for i,d in enumerate(D):
    D[i][1] = int(D[i][1])
#print(D)
D = sorted(D, key=lambda x: x[1],reverse = True)

print(D[0:40])
#find the highest per capita countes
                                         

[['Los Angeles County, CA', 93464], ['Cook County, IL', 69888], ['Maricopa County, AZ', 23682], ['St Louis County, MO', 20662], ['El Paso County, CO', 19044], ['San Diego County, CA', 16624], ['Miami-Dade County, FL', 12443], ['Pinellas County, FL', 5625], ['Multnomah County, OR', 5194], ['Orange County, CA', 5017], ['Lee County, FL', 4007], ['Salt Lake County, UT', 3713], ['Palm Beach County, FL', 3682], ['Bronx County, NY', 3616], ['Broward County, FL', 3554], ['Fulton County, GA', 3159], ['St Louis city, MO', 2530], ['San Bernardino County, CA', 2528], ['Milwaukee County, WI', 2421], ['New Castle County, DE', 2207], ['Pasco County, FL', 2177], ['Jefferson Parish, LA', 2097], ['Duval County, FL', 2059], ['Middlesex County, MA', 2014], ['Greenville County, SC', 1978], ['Suffolk County, NY', 1801], ['Clark County, NV', 1790], ['Spokane County, WA', 1767], ['Brevard County, FL', 1691], ['Hennepin County, MN', 1645], ['Orange County, FL', 1588], ['Hamilton County, OH', 1569], ['Bexar Cou

In [3]:
import csv
import numpy as np
import datetime
import matplotlib.pyplot as plt

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

census_counties = []
census_pop = []
with open('2019Census_Population_Data_by_County.csv') as csvfile: 
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        row[0] = row[0].replace('.','')
        #print(row[0])
#         if(row[0].find('St. Louis') >= 0):
#             print(row[0])
        #print(row[0])
        s = row[0].split(',')
        row[0]=row[0].replace(s[len(s)-1],' '+us_state_abbrev[s[len(s)-1][1:]].lower())
        row[0] = row[0].replace("County,", '')
        census_counties.append(row[0])
        census_pop.append(int(row[len(row)-1].replace(',','')))

persc_rate = []
county = []
fip = []
with open('temphealth.csv') as csvfile: 
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        persc_rate.append(float(row[3]))
        county.append(row[0] +" " +row[1])
        if len(row[2]) == 4:
            fip.append('0'+row[2])
        else:
            fip.append(row[2])
#print(census_counties)
#print(county)

for j,cc in enumerate(census_counties):
    for i,c in enumerate(county):
#         if(c.find('St. Louis')>=0 and cc.find('St. Louis') >= 0):
        state = c.lower().split()
        state2 = cc.lower().split()
        yeet = True
        if len(state) != len(state2):
            continue
        length = len(state)
        if state == state2:
            persc_rate[i] = persc_rate[i]*(census_pop[j]/100) #county death data is deaths per 100,000
            census_counties[j] = 'Done'
            #print(state,state2)

D = []
for i,c in enumerate(county):
    D.append([county[i],float(persc_rate[i]),fip[i]])
#D = np.array(D)
for i,d in enumerate(D):
    D[i][1] = float(D[i][1])
#print(D)
D = sorted(D, key=lambda x: x[1],reverse = True)

print(D[0:40])


with open('worst_counties.csv', mode='w') as worst_counties:
    writer = csv.writer(worst_counties, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for d in D:
        writer.writerow(d)


# f = open("worst_counties.txt", "w")
# for d in D:
#     f.write(d[0] + ", " + str(d[1])+"\n")
# f.close()


[['LOS ANGELES CA', 2750715.318, '06037'], ['MARICOPA AZ', 2179911.204, '04013'], ['HARRIS TX', 1786350.175, '48201'], ['COOK IL', 1673825.725, '17031'], ['CLARK NV', 1205892.3800000001, '32003'], ['WAYNE MI', 1135323.607, '26163'], ['TARRANT TX', 1129050.5550000002, '48439'], ['ORANGE CA', 1073383.896, '06059'], ['DALLAS TX', 1062112.9479999999, '48113'], ['SAN DIEGO CA', 998160.67, '06073'], ['RIVERSIDE CA', 983277.3079999998, '06065'], ['SAN BERNARDINO CA', 906915.36, '06071'], ['BEXAR TX', 883567.314, '48029'], ['KING WA', 806495.9559999999, '53033'], ['HILLSBOROUGH FL', 774255.1680000001, '12057'], ['BROWARD FL', 761583.4199999999, '12011'], ['PALM BEACH FL', 722939.91, '12099'], ['PHILADELPHIA PA', 700156.2880000001, '42101'], ['OAKLAND MI', 687898.4480000001, '26125'], ['DUVAL FL', 679048.295, '12031'], ['OKLAHOMA OK', 674629.164, '40109'], ['FRANKLIN OH', 659694.7559999999, '39049'], ['PINELLAS FL', 655197.3119999999, '12103'], ['MIAMI-DADE FL', 654782.54, '12086'], ['SALT LAKE